## <center> Bayesian Networks </center>
#### <center> David Duffrin </center>

## <center> Bayesian Networks </center>

* Graphical Model
* Probabilistic (conditional)
* Directed
* Acyclic
![Simple Bayes](SimpleBayesNetNodes.png)

## <center> Monty Hall </center>

* There are three doors with one containing a prize
* Three phases:
    1. Choose a door
    2. Monty opens a door that you didn't choose AND that doesn't contain the prize (conditional)
    3. Choose to stay or switch

## <center> Monty's Bayes Net </center>

![Monty Bayes](monty.png)

## <center> Monty Hall Definitions </center>

* Choose Door (C#)
* Monty opens Door (M#)
* Prize Door (P#)

## <center> Monty Hall Solution </center>

1) There is a 1/3 chance that the prize is behind any door and 1/3 chance we choose any door

2) Probabilities of Monty choosing door n:
$$ P(Mn \mid Cn ) = 0 $$
$$ P(Mn \mid \, !Cn ) = \frac{1}{2} $$

3) Probability that the prize is behind stay (x) or switch (y), with the opened door (z):

$$ \color{white}{P(Px \mid Cx, \, Mz) = \frac{P(Mz \mid Cx, \, Px) \, P(Px \mid Cx)}{P(Mz \mid Cx)} = \frac{\frac{1}{2} \cdot \frac{1}{3} }{\frac{1}{2}} = \frac{1}{3}} $$

$$ \color{white}{P(Py \mid Cx, \, Mz) = \frac{P(Mz \mid Cx, \, Py) \, P(Py \mid Cx)}{P(Mz \mid Cx)} = \frac{1 \cdot \frac{1}{3} }{\frac{1}{2}} = \frac{2}{3}} $$

## <center> Computer Science Aspect </center>

* There are packages for graphical models and Bayesian Statistics (pomegranate)
* Allow you to input the states, transitions, and probabilities to generate probabilities for new examples
* Impute most likely outcome
* Fit based on Monte Carlo simulation (or real world data)

## <center> Khan Academy's Problem </center>

* How to estimate Proficiency (Mastery) based on potentially incomplete exercise data
* Proficiency is the likelihood of getting the next problem correct (> threshold)
* Want to minimize the number of problems needed to complete to reach proficiency (so as to not waste time)

## <center> Khan Academy's Subject Structure </center>

## <center> Topic </center>
![Percent Progress](perprog.png)
![Percent Circle](percircle.png)

## <center> Skill </center>
* Mastered
* Level Two
* Level One
* Practiced
* Not Started (NA)

## <center> Exercise </center>
* Correct
* Incorrect
* Not Started (NA)

## <center> Previous Attempts to Estimate Skill </center>

* Counting Not Started Exercises as Incorrect
    * Requires users to complete exercises even if Mastery is implied

* Streak-based model
    * A user that got 10 correct and then 1 incorrect would be seen the same as a user that never started

## <center> Bayesian Attempt to Estimate Skill </center>

* Each exercise independently influences proficiency
* More difficult than Monty Hall because conditional probabilities are unknown
    * Difficult with incomplete data

## <center> Khan's Bayes Net </center>

![Khan Bayes](prof.png)

In [54]:
import pandas as pd
bnet = pd.read_csv('bnet.csv')

In [110]:
bnet.head()

,addition_1,addition_2,addition_3,addition_4,subtraction_1,subtraction_2,subtraction_3,subtraction_4,multiplication_0.5,multiplication_1,...,multiplication_2,multiplication_3,multiplication_4,number_line,number_line_2,number_line_3,exponents_1,exponents_2,exponents_3,exponents_4
0,0,0,0,0,0,0,0,0,-1,0,...,0,0,0,-1,-1,-1,0,0,0,-1
1,0,-1,1,0,1,0,-1,-1,-1,0,...,1,1,1,1,-1,-1,-1,-1,-1,-1
2,0,0,0,-1,0,0,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1,1,1,1,1,1,1,1,-1,1,...,1,1,-1,-1,-1,-1,0,-1,-1,-1
4,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## <center> Problem </center>

* Proficiency is a hidden variable

## <center> Solution </center>

* Expectation Maximization algorithm to impute the topic proficiency based on the given exercise data

## <center> EM Algorithm </center>

An iterative method for finding maximum likelihood estimates with two steps:

* Expectation - impute hidden node (categorize points - clustering)

* Maximization - computes parameters maximizing the expected log-likelihood
    * Adjusts the conditional probabilities of the exercise and proficiency

## <center> EM Example </center>

![EM Algorithm](EM.gif)

In [105]:
from numpy import asmatrix, asarray, ones, zeros, mean, sum, arange, prod, dot, loadtxt
from numpy.random import random, randint
#import pickle

MISSING_VALUE = -1 # a constant I will use to denote missing integer values

# Impute hidden T - Expectation
def impute_hidden_node(E, I, theta, sample_hidden):

    theta_T, theta_E = theta

    # calculate the unnormalized probability associated with the hidden unit being a 0
    theta_E_wide = asarray( ones([E.shape[0],1]) * asmatrix(theta_E[:,0]) )
    p_vis_0 = I * (theta_E_wide * E + (1-theta_E_wide) * (1-E)) + (I==0)*1
    prob_0_unnorm = (1-theta_T) * prod(p_vis_0, 1)

    # calculate the unnormalized probability associated with the hidden unit being a 1
    theta_E_wide = asarray( ones([E.shape[0],1]) * asmatrix(theta_E[:,1]) )
    p_vis_1 = I * (theta_E_wide * E + (1-theta_E_wide) * (1-E)) + (I==0)*1
    prob_1_unnorm = theta_T * prod(p_vis_1, 1)

    hidden = prob_1_unnorm / (prob_0_unnorm + prob_1_unnorm)

    if sample_hidden:
        # set the hidden unit to a 0 or 1 instead of a probability of activation
        hidden = (hidden > random( hidden.shape ))*1

    return hidden

# Create data that follows the given distribution
def simulate(theta, nsamples):

    theta_T, theta_E = theta
    
    T = (theta_T > random(nsamples))

    # (multiplying by T selects the cases where T=1, multiplying by 1-T selects the cases where T=0)
    E = (asmatrix(1-T).transpose() * theta_E[:,0] > random([nsamples, theta_E.shape[0]]))  \
      + (asmatrix(T).transpose() * theta_E[:,1] > random([nsamples, theta_E.shape[0]]))

    E = asarray(E * 1)

    return T, E

# Calculate conditional probabilities (E Thetas) by counting
def compute_theta(T, E):
    
    theta_T = mean(T) # the probability is the average activation
    theta_E = zeros([E.shape[1], 2]) 

    for e in range(E.shape[1]):
        E_col = E[:,e] 
        ix = E_col != MISSING_VALUE  # row indices that are not missing this evidence
        theta_E[e,0] = sum( E_col[ix] * (1-T[ix]) ) / float( sum( 1-T[ix] ) ) # the average of E when T=0
        theta_E[e,1] = sum( E_col[ix] * T[ix] ) / float( sum( T[ix] ) ) # the average of E when T=1

    return [theta_T, theta_E]

# Print the T and E Thetas
def print_theta(theta):

    theta_T, theta_E = theta
    
    print("T\t0: %f\t1:%f" % (1-theta_T, theta_T))
    for i in range( theta_E.shape[0] ):
        print("E%d T=0\t0: %f\t1:%f" % (i, 1-theta_E[i,0], theta_E[i,0]))
        print("E%d T=1\t0: %f\t1:%f" % (i, 1-theta_E[i,1], theta_E[i,1]))

# Iterate through the Expectation Maximization algorithm
def learn(T, E, max_iter, sample_hidden):
    
    I = (E!=MISSING_VALUE)*1  #indicator matrix on whether evidence for each E-variable is present
    
    theta = compute_theta( T,E ) 

    for i in range(max_iter):
        T = impute_hidden_node(E, I, theta, sample_hidden)  # E-step

        # there are two equivalent solutions with T=0 and T=1 flipped.  always take the solution where T=1 is more probable.
        if mean(T) < 0.5:
            T = 1-T

        theta = compute_theta( T, E )    # M-step

        print("Run %d produced theta of:" % i)
        print_theta(theta) 
    return theta

# Create a known distribution to test convergence and EM algorithm
def simulated_example():

    # start by specifying a TRUE distributions, theta.
    theta_T = 0.75 # probability that T is 1
    theta_E = asarray(zeros( [5, 2] )) # probability that E is 1.  [number of leaves] x [number of T states]
    theta_E[0,0] = 0.55 # probability that E0 = 1 if T = 0 
    theta_E[0,1] = 0.95 # probability that E0 = 1 if T = 1
    theta_E[1,0] = 0.60 # probability that E1 = 1 if T = 0
    theta_E[1,1] = 0.95 # probability that E1 = 1 if T = 1
    theta_E[2,0] = 0.24 # probability that E2 = 1 if T = 0
    theta_E[2,1] = 0.42 # probability that E2 = 1 if T = 1
    theta_E[3,0] = 0.13 # probability that E3 = 1 if T = 0
    theta_E[3,1] = 0.72 # probability that E3 = 1 if T = 1
    theta_E[4,0] = 0.62 # probability that E4 = 1 if T = 0
    theta_E[4,1] = 0.66 # probability that E4 = 1 if T = 1
    theta = [theta_T, theta_E]
    
    # now generate/simulate a dataset according to theta 
    row_count = 10000;  print("rowcount = %d" % row_count)
    [T, E] = simulate(theta, row_count)
    
    # randomize/hide the 'T' variable, to see if we can re-learn it
    T2 = T.copy()
    T = randint(2, size=row_count)
    
    # in addition, randomly remove between 1 to 3 E-values for each sample as 'missing' data
    for i in range(3):
        E[arange(row_count), randint(5,size=row_count)] = MISSING_VALUE
    
    # finally, try to learn the parameters 
    theta_learned = learn(T, E, 400, sample_hidden=True)
    
    print('Starting State:')
    print_theta(compute_theta(T,E))
    print('Ending State:')
    print_theta(theta_learned)
    print('Goal:')
    print_theta(theta)

# The real deal. Test on KA data.
def ka_data_example():
    global theta_learned
    E = loadtxt('bnet.csv', dtype=int, delimiter=',', skiprows=1)
    T = randint(2, size=E.shape[0])

    theta_learned = learn(T, E, 200, sample_hidden=True)

    print('Starting State:')
    print_theta(compute_theta(T,E))
    print('Ending State:')
    print_theta(theta_learned)
    #pickle.dump([theta_learned[0], theta_learned[1].tolist()], open('theta.pickle', 'wb'))

## <center> Code Overview </center>

Originally starts with random conditional probabilities and proficiency probabilities and slowly converges toward the real values.

1) Reads in data (or simulates data with NA spaces)

2) Learn Phase - iterates X times through the EM cycle

* Expectation - infers proficiency for each student based on current conditional probabilities

* Maximization - recalculates the conditional probabilities based on the new proficiencies
    * New probabilities are used by the expectation phase on the next pass

3) Prints results

In [106]:
simulated_example()

rowcount = 10000
Run 0 produced theta of:
T	0: 0.499400	1:0.500600
E0 T=0	0: 0.146072	1:0.853928
E0 T=1	0: 0.142691	1:0.857309
E1 T=0	0: 0.124235	1:0.875765
E1 T=1	0: 0.144984	1:0.855016
E2 T=0	0: 0.599285	1:0.400715
E2 T=1	0: 0.631167	1:0.368833
E3 T=0	0: 0.424098	1:0.575902
E3 T=1	0: 0.412311	1:0.587689
E4 T=0	0: 0.343318	1:0.656682
E4 T=1	0: 0.368442	1:0.631558
Run 1 produced theta of:
T	0: 0.498900	1:0.501100
E0 T=0	0: 0.146130	1:0.853870
E0 T=1	0: 0.142636	1:0.857364
E1 T=0	0: 0.128426	1:0.871574
E1 T=1	0: 0.140398	1:0.859602
E2 T=0	0: 0.593147	1:0.406853
E2 T=1	0: 0.637584	1:0.362416
E3 T=0	0: 0.422895	1:0.577105
E3 T=1	0: 0.413417	1:0.586583
E4 T=0	0: 0.334630	1:0.665370
E4 T=1	0: 0.376739	1:0.623261
Run 2 produced theta of:
T	0: 0.499700	1:0.500300
E0 T=0	0: 0.146389	1:0.853611
E0 T=1	0: 0.142292	1:0.857708
E1 T=0	0: 0.141910	1:0.858090
E1 T=1	0: 0.127161	1:0.872839
E2 T=0	0: 0.633006	1:0.366994
E2 T=1	0: 0.597546	1:0.402454
E3 T=0	0: 0.429132	1:0.570868
E3 T=1	0: 0.407190	1:0.

In [67]:
ka_data_example()

Run 0 produced theta of:
T	0: 0.495262	1:0.504738
E0 T=0	0: 0.011228	1:0.988772
E0 T=1	0: 0.012823	1:0.987177
E1 T=0	0: 0.061257	1:0.938743
E1 T=1	0: 0.061262	1:0.938738
E2 T=0	0: 0.122558	1:0.877442
E2 T=1	0: 0.127266	1:0.872734
E3 T=0	0: 0.141994	1:0.858006
E3 T=1	0: 0.107054	1:0.892946
E4 T=0	0: 0.009379	1:0.990621
E4 T=1	0: 0.009736	1:0.990264
E5 T=0	0: 0.077604	1:0.922396
E5 T=1	0: 0.065708	1:0.934292
E6 T=0	0: 0.141960	1:0.858040
E6 T=1	0: 0.141371	1:0.858629
E7 T=0	0: 0.159601	1:0.840399
E7 T=1	0: 0.153273	1:0.846727
E8 T=0	0: 0.015447	1:0.984553
E8 T=1	0: 0.015235	1:0.984765
E9 T=0	0: 0.029599	1:0.970401
E9 T=1	0: 0.033704	1:0.966296
E10 T=0	0: 0.020386	1:0.979614
E10 T=1	0: 0.024038	1:0.975962
E11 T=0	0: 0.172879	1:0.827121
E11 T=1	0: 0.165637	1:0.834363
E12 T=0	0: 0.176902	1:0.823098
E12 T=1	0: 0.175537	1:0.824463
E13 T=0	0: 0.262967	1:0.737033
E13 T=1	0: 0.220049	1:0.779951
E14 T=0	0: 0.055528	1:0.944472
E14 T=1	0: 0.055830	1:0.944170
E15 T=0	0: 0.098894	1:0.901106
E15 T=1	0

## <center> Example User </center>

* Let's look at an example of if a user answers E20 correctly and no other exercise
* Note: If we put in more exercises, the formula becomes very long
$$ P(T_1 \mid E20_1) = \frac{P(E20_1 \mid T_1) \, P(T_1)}{P(E20_1)} = \frac{0.726852 \cdot 0.764442 }{0.764442 \cdot 0.726852 + 0.235558 \cdot 0.217391} = 0.915616 $$

## <center> Validation </center>

![Coefficients](coef.png)

## <center> Khan's Bayes Net </center>

![Khan Bayes](prof.png)

## <center> Khan Academy's Other Problems </center>

* Best ordering of topics to minimize time and maximize success
    * Originally based off of the school curriculum
* What software intervention would help the student
    * ie. Wrong answer on an exercise, so the website displays a video to rewatch that will help

## <center> Sources </center>

http://derandomized.com/post/20009997725/bayes-net-example-with-python-and-khanacademy

https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm

https://github.com/jmschrei/pomegranate

